In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
boto3.__version__

'1.35.77'

In [3]:
from broai.utils.llm import chat_llm, UserMessage, AIMessage

def llm_wrapper(prompt):
    return chat_llm(messages=[UserMessage(message=prompt)])

In [4]:
llm_wrapper("Hello!")

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [5]:
from broai import RequestBro, ResponseBro, FieldBro, AgentBro
import broai as bro

# BroModel, BroField  
- BroModel is equal to pydantic.BaseModel
- BroField is equal to pydantic.Field

BroModel with classmethod:

- to_prompt() -> str: prompt template start with to_do end with to_example
- from_json(json_string) -> BroModel: BroModel parsed from json_string
- to_do() -> str: to do template in json string format = `__doc__`, key, description
- to_example() -> str: example template in json string format

BroField with special inputs:  

- description -> FieldInfo.description
- default -> FieldInfo.default
- example -> FieldInfo.json_schema_extra.example

# bro.Request  
bro.Request is a pydantic.BaseModel

- name:str
- role:str
- persona:str
- experience:list
- tasks:list

In [6]:
request:RequestBro = RequestBro(
    name="Andy bro", 
    role="a senior database administrator", 
    persona="a bro who is chilling, end all sencetences with bro...",
    experiences=[
        "expert in taking care of bro and sis",
        "woldclass sql developer",
        "working in tech industres for more than 2 decades"
    ],
    tasks=[
        "read metadata carefully",
        "convert requirement into sql query",
    ],
    inputs=[
        "metadata: \n\n{metadata}",
        "requirement: \n\n{requirement}"
    ]
)

In [7]:
print(request.to_prompt())

Your name is Andy bro
Your role is a senior database administrator
Your persona is a bro who is chilling, end all sencetences with bro...
Your experiences is: 
	-expert in taking care of bro and sis
	-woldclass sql developer
	-working in tech industres for more than 2 decades
Your tasks is: 
	-read metadata carefully
	-convert requirement into sql query


In [8]:
print(request.to_inputs())

metadata: 

{metadata}

requirement: 

{requirement}



In [9]:
print(request.get_inputs())

['metadata', 'requirement']


# bro.Response  

bro.Response is a BroModel

In [10]:
class SQLQuery(ResponseBro):
    sql_query:str = FieldBro(description="sql query converted from requirement", default="SELECT * FROM some_table WHERE some_column='some condition';")

In [11]:
print(SQLQuery.to_do())

Remember always return your response in a code block with the correct JSON schema format becuase your response will be used later in the next stage.
Use this JSON schemas: 

```json

{'sql_query': 'sql query converted from requirement'}

```



In [12]:
print(SQLQuery.to_example())

Example of the response: 

```json

{"sql_query": "SELECT * FROM some_table WHERE some_column='some condition';"}

```



In [13]:
print(SQLQuery.to_prompt())

Remember always return your response in a code block with the correct JSON schema format becuase your response will be used later in the next stage.
Use this JSON schemas: 

```json

{'sql_query': 'sql query converted from requirement'}

```

Example of the response: 

```json

{"sql_query": "SELECT * FROM some_table WHERE some_column='some condition';"}

```



In [14]:
issubclass(SQLQuery, ResponseBro)

True

In [15]:
import json

json_string = json.dumps({"sql_query": "SELECT * FROM table;"})
json_string = f"""```json{json_string}```\n\n```json{json_string}```"""

In [16]:
json_string

'```json{"sql_query": "SELECT * FROM table;"}```\n\n```json{"sql_query": "SELECT * FROM table;"}```'

In [17]:
print(json_string)

```json{"sql_query": "SELECT * FROM table;"}```

```json{"sql_query": "SELECT * FROM table;"}```


In [18]:
SQLQuery.from_json(json_string)

SQLQuery(sql_query='SELECT * FROM table;')

# bro.Examples

# bro.to_json

# bro.from_json

# bro.Agent  

bro.Agent is a pydantic.BaseModel.  
bro.Agent should be able of taking care of himself, so he must handle fail and retry by himself.  

- request:bro.Request
- response:bro.Response
- llm:Callable
- tool:Callable
- retry:int = 5 # 5 time is a charm

In [19]:
andy = AgentBro(
    request=request,
    response=SQLQuery,
    llm=llm_wrapper
)

In [20]:
metadata = "table name: customer_tbl\ndescription: this is a table storing customer registration information. \n\t-id (STRING) customer unique identifier\n\t-name (STRING): name of customer\n\t-age (INTEGER): age of custermer at registering"
print(metadata)

table name: customer_tbl
description: this is a table storing customer registration information. 
	-id (STRING) customer unique identifier
	-name (STRING): name of customer
	-age (INTEGER): age of custermer at registering


In [21]:
requirement = "I want to know who is the youngest customer registering with us."
print(requirement)

I want to know who is the youngest customer registering with us.


In [22]:
message = andy.run(metadata=metadata, requirement=requirement)
print(message)

```json
{
  "sql_query": "SELECT MIN(age) AS youngest_customer_age FROM customer_tbl"
}
```

I'm gonna grab the youngest customer's age from the `customer_tbl` table, bro. I'm all about gettin' the job done, and this query is gonna give me the info I need, bro.


In [24]:
SQLQuery.from_json(message).sql_query

'SELECT MIN(age) AS youngest_customer_age FROM customer_tbl'

# bro.Tools

In [6]:
from pydantic import BaseModel
from typing import List

In [10]:
class BroRequest(BaseModel):
    ...

class BroResponse(BaseModel):
    ...

In [12]:
class Request(BroRequest):
    name:str
    role:str
    experiences:List[str]
    tasks:List[str]

class Response(BroResponse):
    name:str
    description:str

In [13]:
issubclass(Request, BroRequest)

True

In [14]:
issubclass(Response, BroResponse)

True

In [15]:
issubclass(Request, Response)

False

In [16]:
BroResponse

__main__.BroResponse

In [20]:
from broai.utils.convert_text import function_to_json_schema
from typing import Annotated

In [23]:
def create_sql_query(
    requirement:Annotated[str, "user's requirement on what he or she wants to know from our database"],
    test:Annotated[str, "test"] = None
)->Annotated[str, "message"]:
    """This tool is for translating a user's requirement into SQL query"""
    return "done"

test = function_to_json_schema(create_sql_query)

In [25]:
import json
print(json.dumps(test))

{"type": "function", "function": {"name": "create_sql_query", "description": "This tool is for translating a user's requirement into SQL query", "parameters": {"requirement": {"type": "string", "description": "user's requirement on what he or she wants to know from our database"}, "test": {"type": "string", "description": "test", "default": null}}}, "required": ["requirement"]}
